In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.style import use; use('/Users/stray/.matplotlib/gruvbox.mplstyle')

from prepare import initial_prep, prep_driver_data, prep_vehicle_data, prep_damage_data, create_fault_factor_cols, clean_traffic_cats, clean_weather_cats

In [2]:
# read in CSV to DataFrame
df = pd.read_csv('~/codeup-data-science/project_danger_zone/all_accident_data.csv')
df = initial_prep(df)
df = prep_driver_data(df)
df = prep_vehicle_data(df)
df = prep_damage_data(df)
df = create_fault_factor_cols(df)
df = clean_traffic_cats(df)
df = clean_weather_cats(df)

In [80]:
df = df[[
    'num_of_injuries',
    'num_of_vehicles',
    'num_of_occupants',
    'car_contained_injury'
]]

In [4]:
print(df.head(1).T)

                                                                      0
crash_url             https://app.myaccident.org/accident/0000281836...
crash_id                                                       18365140
crash_date                                             07/12/2021 19:00
crash_latitude                                                29.416846
crash_longitude                                              -98.409521
accident_factor                                 new flyer turned unsafe
speed_limit_mph                                                      30
num_of_injuries                                                       0
num_of_vehicles                                                       2
num_of_occupants                                                      2
at_fault                                                              1
car_contained_injury                                                  0
vehicle_make                                                    